In [11]:
import torch
import imageio
from pathlib import Path
import sys
import os
import math
base_path = Path(os.path.abspath('')).parent
sys.path.append(str(base_path))

import matplotlib.pyplot as plt
def disp_im(im):
    plt.close()
    plt.cla()
    fig = plt.imshow(im.copy())
    plt.close()
    display(fig.figure)

In [4]:
from dataLoader.ray_utils import get_ray_directions, get_rays
from renderer import chunk_renderer, BundleRender
import numpy as np

w = 800
h = 800
camera_angle_x = 0.6194058656692505

fx = 0.5 * w / np.tan(0.5 * camera_angle_x)  # original focal length
fy = fx

directions = get_ray_directions(h, w, [fx,fy])  # (h, w, 3)
directions = directions / torch.norm(directions, dim=-1, keepdim=True)
c2w = torch.eye(4)
blender2opencv = torch.tensor([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]]).float()

c2w = torch.tensor([
    [
        -0.9999021291732788,
        0.004192245192825794,
        -0.013345719315111637,
        -0.05379832163453102
    ],
    [
        -0.013988680206239223,
        -0.2996590733528137,
        0.95394366979599,
        3.845470428466797
    ],
    [
        -4.6566125955216364e-10,
        0.9540371894836426,
        0.29968830943107605,
        1.2080823183059692
    ],
    [
        0.0,
        0.0,
        0.0,
        1.0
    ]
]) @ blender2opencv
# c2w = torch.tensor([
#     [
#         0.24868656694889069,
#         -0.6090989112854004,
#         0.7530957460403442,
#         3.0358264446258545
#     ],
#     [
#         0.9685839414596558,
#         0.1563878208398819,
#         -0.1933593899011612,
#         -0.7794567346572876
#     ],
#     [
#         -7.450580152834618e-09,
#         0.7775223851203918,
#         0.6288550496101379,
#         2.5349960327148438
#     ],
#     [
#         0.0,
#         0.0,
#         0.0,
#         1.0
#     ]
# ]) @ blender2opencv

c2w = torch.tensor([
    [
        0.8443264365196228,
        -0.2852588891983032,
        0.45358625054359436,
        1.8284648656845093
    ],
    [
        0.5358295440673828,
        0.4494929313659668,
        -0.7147326469421387,
        -2.8811798095703125
    ],
    [
        1.4901164746561335e-08,
        0.8465123176574707,
        0.5323687791824341,
        2.146047353744507
    ],
    [
        0.0,
        0.0,
        0.0,
        1.0
    ]
]) @ blender2opencv


# c2w = torch.tensor([
#     [
#         -0.24868960678577423,
#         0.6655282974243164,
#         -0.7037226557731628,
#         -2.836796998977661
#     ],
#     [
#         -0.9685831665992737,
#         -0.17087841033935547,
#         0.18068504333496094,
#         0.7283647656440735
#     ],
#     [
#         0.0,
#         0.7265485525131226,
#         0.6871150732040405,
#         2.7698497772216797
#     ],
#     [
#         0.0,
#         0.0,
#         0.0,
#         1.0
#     ]
# ]) @ blender2opencv

# c2w = torch.tensor([
#     [
#         0.6845461130142212,
#         -0.24688588082790375,
#         0.6858890652656555,
#         2.7649075984954834
#     ],
#     [
#         0.7289695143699646,
#         0.23184067010879517,
#         -0.644091010093689,
#         -2.596414089202881
#     ],
#     [
#         -1.4901159417490817e-08,
#         0.9409023523330688,
#         0.33867788314819336,
#         1.365254282951355
#     ],
#     [
#         0.0,
#         0.0,
#         0.0,
#         1.0
#     ]
# ]) @ blender2opencv




rays_o, rays_d = get_rays(directions, c2w)  # both (h*w, 3)

rays = torch.cat([rays_o, rays_d], 1)


In [25]:
# pano = imageio.imread('../log/car_v38_freezelayers_randbg/imgs_test_all/envmaps/pano.exr')
pano = imageio.imread('../log/noprednorms_nl0_conserve_pb0/car_v38/imgs_test_all/envmaps/mapped_pano.png')
pano = torch.as_tensor(pano)

/tmp/ipykernel_26212/2296568613.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  pano = imageio.imread('../log/noprednorms_nl0_conserve_pb0/car_v38/imgs_test_all/envmaps/mapped_pano.png')


In [15]:
# line sphere intersection
a = (rays_d * rays_d).sum(dim=-1, keepdim=True)
b = 2 * (rays_d * rays_o).sum(dim=-1, keepdim=True)
c = (rays_o * rays_o).sum(dim=-1, keepdim=True) - 1
sol1 = (-b + (b**2 - 4 * a * c).sqrt()) / 2 / a
sol2 = (-b - (b**2 - 4 * a * c).sqrt()) / 2 / a
dist = torch.minimum(sol1, sol2)
mask = ~torch.isnan(dist).reshape(-1)
pt = rays_o + dist * rays_d

# convert to equirectangular

a, b, c = pt.T
norm = torch.sqrt(a**2+b**2)
phi = torch.atan2(b, a)
theta = torch.atan2(c, norm)


In [41]:
base = Path('../rotating_ball')
for i, offset in enumerate(torch.linspace(0, 2*math.pi, 200)):
    c1 = ((phi+offset) % (2*math.pi) - math.pi) / math.pi
    c2 = -theta/math.pi*2
    pH, pW = pano.shape[0], pano.shape[1]
    col = ((c1 + 1)/2 * pW).clip(1, pW-1).long()
    row = ((c2 + 1)/2 * pH).clip(1, pH-1).long()
    im = torch.zeros((h*w, 4), dtype=pano.dtype)

    LdotN = (rays_d * -pt).sum(dim=-1, keepdim=True).clip(min=0)
    im[mask, :3] = (pano[row[mask], col[mask]].float() * LdotN[mask]).byte()
    im[mask, 3] = 255
    im = im.reshape(h, w, 4)
    # disp_im(im[..., :3].numpy())
    imageio.imwrite(base / f'{i:03d}.png', im.numpy())